In [1]:
fileNameQrd = 'qrd_canonical_model.csv'
fileNameMatchRuleBook = 'ruleDict.json'
fileNameDocumentTypeNames = 'documentTypeNames.json'

fsMountName = '/mounted'

In [2]:
import pprint
import pandas as pd
import uuid
import json
import os
import glob
import re
import sys
from bs4 import NavigableString, BeautifulSoup
from collections import defaultdict
import random
import string

from utils.config import config
from utils.logger.logger import loggerCreator

# ePI Modules
from parse.rulebook.rulebook import StyleRulesDictionary

from parse.extractor.parser import parserExtractor
from match.matchDocument.matchDocument import MatchDocument
from documentAnnotation.documentAnnotation import DocumentAnnotation
from htmlDocTypePartitioner.partition import DocTypePartitioner
from extractContentBetweenHeadings.dataBetweenHeadingsExtractor import DataBetweenHeadingsExtractor
from fhirXmlGenerator.fhirXmlGenerator import FhirXmlGenerator
from fhirService.fhirService import FhirService
from utils.logger.matchLogger import MatchLogger
from languageInfo.documentTypeNames.documentTypeNames import DocumentTypeNames


class FolderNotFoundError(Exception):
    pass


def getRandomString(N):
    str_ = ''.join(random.choice(string.ascii_uppercase + string.digits
                                 + string.ascii_lowercase) for _ in range(N))
    return str_


def convertHtmlToJson(domain, procedureType, languageCode, fileNameHtml, fileNameQrd, fileNameLog, fsMountName, localEnv):

    if localEnv is True:
        pathSep = "\\"
        module_path = os.path.join(os.path.abspath(os.path.join('..')), 'data', 'converted_to_html', f'{domain}', f'{procedureType}', f'{languageCode}')
    else:
        pathSep = "/"
        module_path = os.path.join(f'{fsMountName}', 'data', 'converted_to_html', f'{domain}', f'{procedureType}', f'{languageCode}')
    
    # Generate output folder path
    output_json_path = module_path.replace('converted_to_html', 'outputJSON')

    """
        Check if input folder exists, else throw exception
    """
    if(os.path.exists(module_path)):
        filenames = glob.glob(os.path.join(module_path, fileNameHtml))

        # Create language specific folder in outputJSON folder if it doesn't exist
        if(not os.path.exists(output_json_path)):
            os.mkdir(output_json_path)
        logger = MatchLogger(f'Parser_{getRandomString(1)}', fileNameHtml,
                             domain, procedureType, languageCode, "HTML", fileNameLog)

        styleLogger = MatchLogger(
            f'Style Dictionary_{getRandomString(1)}', fileNameHtml, domain, procedureType, languageCode, "HTML", fileNameLog)

        styleRulesObj = StyleRulesDictionary(styleLogger,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType,
                                             fsMountName=fsMountName,
                                             localEnv=localEnv)

        parserObj = parserExtractor(config, logger, styleRulesObj.styleRuleDict,
                                    styleRulesObj.styleFeatureKeyList,
                                    styleRulesObj.qrd_section_headings)

        for input_filename in filenames:
          # if(input_filename.find('Kalydeco II-86-PI-clean')!=-1):
            output_filename = input_filename.replace('converted_to_html', 'outputJSON')
            style_filepath =  output_filename.replace('.html','.txt')
            style_filepath =  style_filepath.replace('.txtl','.txt')
            style_filepath =  style_filepath.replace('.htm','.txt')
            print("-------------",style_filepath,"-----------------")

            output_filename = output_filename.replace('.html', '.json')
            output_filename = output_filename.replace('.htm', '.json')
            print(input_filename, output_filename)
            parserObj.createPIJsonFromHTML(input_filepath=input_filename,
                                           output_filepath=output_filename,
                                           style_filepath = style_filepath,
                                           img_base64_dict=parserObj.convertImgToBase64(input_filename)
                                           )
            
        return output_filename.split(pathSep)[-1], style_filepath
    else:
        try:    
            raise FolderNotFoundError(module_path + " not found")
        except:  
            logger.logFlowCheckpoint("Folder For Language Code Not Found In Input File")
            logger.logException("Folder For Language Code Not Found In Input File")
        raise FolderNotFoundError(module_path + " not found")
        return None


def splitJson(domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog, fsMountName, localEnv):

    styleLogger = MatchLogger(
        f'Style Dictionary_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    styleRulesObj = StyleRulesDictionary(styleLogger,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType,
                                             fsMountName=fsMountName,
                                             localEnv=localEnv)
    if localEnv is True:
        path_json = os.path.join(os.path.abspath(os.path.join(
                '..')), 'data', 'outputJSON', domain, procedureType, languageCode, fileNameJson)
    else:
        path_json = os.path.join(f'{fsMountName}', 'data', 'outputJSON', domain,  procedureType, languageCode, fileNameJson)
        
    partitionLogger = MatchLogger(
        f'Partition_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    partitioner = DocTypePartitioner(partitionLogger, localEnv)

    partitionedJsonPaths = partitioner.partitionHtmls(
        styleRulesObj.qrd_section_headings, path_json)

    return partitionedJsonPaths


def extractAndValidateHeadings(domain,
                               procedureType,
                               languageCode,
                               documentNumber,
                               fileNameDoc,
                               fileNameQrd,
                               fileNameMatchRuleBook,
                               fileNameDocumentTypeNames,
                               fileNameLog,
                               stopWordFilterLen=6,
                               isPackageLeaflet=False,
                               medName=None,
                               fsMountName='/mounted',
                               localEnv=False):

    if documentNumber == 0:
        topHeadingsConsidered = 4
        bottomHeadingsConsidered = 6
    elif documentNumber == 1:
        topHeadingsConsidered = 3
        bottomHeadingsConsidered = 5
    elif documentNumber == 2:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 15
    else:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 10

    print(f"Starting Heading Extraction For File :- {fileNameDoc}")
    logger = MatchLogger(f"Heading Extraction {fileNameDoc}", fileNameDoc, domain, procedureType, languageCode, documentNumber, fileNameLog)
    logger.logFlowCheckpoint("Starting Heading Extraction")

    stopWordlanguage = DocumentTypeNames(
        fileNameDocumentTypeNames=fileNameDocumentTypeNames,
        languageCode=languageCode,
        domain=domain,
        procedureType=procedureType,
        documentNumber=documentNumber,
        fsMountName=fsMountName,
        localEnv=localEnv).extractStopWordLanguage()

    matchDocObj = MatchDocument(
        logger,
        domain,
        procedureType,
        languageCode,
        documentNumber,
        fileNameDoc,
        fileNameQrd,
        fileNameMatchRuleBook,
        fileNameDocumentTypeNames,
        topHeadingsConsidered,
        bottomHeadingsConsidered,
        stopWordFilterLen,
        stopWordlanguage,
        isPackageLeaflet,
        medName,
        fsMountName,
        localEnv)
    df, coll = matchDocObj.matchHtmlHeaddingsWithQrd()

    return df, coll


def parseDocument(htmlDocPath, fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, fsMountName = '/mounted', localEnv= False, medName = None):
    
    if localEnv is True:
        pathSep = "\\"
        fileNameLog = os.path.join(os.path.abspath(os.path.join('..')),'data','FinalLog.txt')
    else:
        pathSep = "/"
        fileNameLog = os.path.join(f'{fsMountName}','data','FinalLog.txt')
    pathComponents = htmlDocPath.split(pathSep)
    print(pathComponents)
    fileNameHtml = pathComponents[-1]
    languageCode =  pathComponents[-2]
    procedureType = pathComponents[-3]
    domain = pathComponents[-4]

    #procedureType = "CAP"
    print(fileNameHtml,languageCode,procedureType)
    
    
    
    flowLogger =  MatchLogger(f"Flow Logger HTML_{getRandomString(1)}", fileNameHtml, domain, procedureType, languageCode, "HTML", fileNameLog)

    flowLogger.logFlowCheckpoint("Starting HTML Conversion To Json")
    ###Convert Html to Json
    fileNameJson, stylesFilePath = convertHtmlToJson( domain, procedureType, languageCode, fileNameHtml, fileNameQrd, fileNameLog, fsMountName, localEnv)
    
    flowLogger.logFlowCheckpoint("Completed HTML Conversion To Json")

    flowLogger.logFlowCheckpoint("Starting Json Split")

    ###Split Uber Json to multiple Jsons for each category.
    partitionedJsonPaths = splitJson(domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog, fsMountName, localEnv)
    
    partitionedJsonPaths = [ path.split(pathSep)[-1] for path in partitionedJsonPaths]
    flowLogger.logFlowCheckpoint(str(partitionedJsonPaths))
    
    flowLogger.logFlowCheckpoint("Completed Json Split")
    
    flowLogger.logFlowCheckpoint("Started Processing Partitioned Jsons")
    
    for index, fileNamePartitioned in enumerate(partitionedJsonPaths):
        
        flowLogger.logFlowCheckpoint(f"\n\n\n\n||||||||||||||||||||||||||||||||{str(index)} ||||| {str(fileNamePartitioned)}||||||||||||||||||||||||||||||||\n\n\n\n")
        
        if index == 3:
            stopWordFilterLen = 100
            isPackageLeaflet = True
        else:
            stopWordFilterLen = 6
            isPackageLeaflet = False
            
        df, coll = extractAndValidateHeadings(domain,
                                    procedureType,
                                    languageCode,
                                    index,
                                    fileNamePartitioned,
                                    fileNameQrd,
                                    fileNameMatchRuleBook,
                                    fileNameDocumentTypeNames,
                                    fileNameLog,
                                    stopWordFilterLen=stopWordFilterLen,
                                    isPackageLeaflet=isPackageLeaflet,
                                    medName=medName,
                                    fsMountName=fsMountName,
                                    localEnv=localEnv)
        
        
        print(f"Completed Heading Extraction For File")
        flowLogger.logFlowCheckpoint("Completed Heading Extraction For File")
        
        print(f"Starting Document Annotation For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Document Annotation For File")
        documentAnnotationObj = DocumentAnnotation(fileNamePartitioned,'c20835db4b1b4e108828a8537ff41506','https://spor-sit.azure-api.net/pms/api/v2/',df,coll)
        try:
            pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
            print(pms_oms_annotation_data)
        except:
            pms_oms_annotation_data = None
            print("Error Found")
            
        print(f"Completed Document Annotation")        
        flowLogger.logFlowCheckpoint("Completed Document Annotation")
        
        print(f"Starting Extracting Content Between Heading For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Extracting Content Between Heading")
        
        extractContentlogger =  MatchLogger(f'ExtractContentBetween_{index}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, coll, domain, procedureType, languageCode, fsMountName, localEnv)
        dfExtractedHierRR = extractorObj.extractContentBetweenHeadings(fileNamePartitioned)
        
        print(f"Completed Extracting Content Between Heading")        
        flowLogger.logFlowCheckpoint("Completed Extracting Content Between Heading")
        
        xmlLogger =  MatchLogger(f'XmlGeneration_{index}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger, pms_oms_annotation_data, stylesFilePath, medName, fsMountName, localEnv)
        fileNameXml = fileNamePartitioned.replace('.json','.xml')
        generatedXml = fhirXmlGeneratorObj.generateXml(dfExtractedHierRR, fileNameXml)
        
        fhirServiceLogger =  MatchLogger(f'XML Submission Logger_{index}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)

        fhirServiceObj = FhirService(fhirServiceLogger, generatedXml, fsMountName, localEnv)
        fhirServiceObj.submitFhirXml()
        print(f"Created XML File For :- {fileNamePartitioned}")        

        #return df,coll,dfExtractedHierRR
    
    flowLogger.logFlowCheckpoint("Completed Processing Partitioned Jsons")


C:\Users\vipsharm\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vipsharm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
parseDocument("F:\\Projects\\EMA\\Repository\\EMA EPI PoC\\function_code\\data\\converted_to_html\\H\\CAP\\en\\Adakveo-4874 EN PI.html", fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, medName = "Aerius", fsMountName = '/mounted', localEnv= True)

2021-05-07 18:47:12,775 : Flow Logger HTML_K : Starting HTML Conversion To Json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-07 18:47:12,786 : Style Dictionary_L : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-07 18:47:12,837 : Style Dictionary_L : Qrd Section Keys Retrieved For Style Dictionary: ANNEX I, ANNEX II, ANNEX III, B. PACKAGE LEAFLET | H | CAP |  en | HTML | Adakveo-4874 EN PI.html


['F:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'data', 'converted_to_html', 'H', 'CAP', 'en', 'Adakveo-4874 EN PI.html']
Adakveo-4874 EN PI.html en CAP
------------- F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.txt -----------------
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\converted_to_html\H\CAP\en\Adakveo-4874 EN PI.html F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.json


2021-05-07 18:47:13,236 : Parser_C : Style Information Stored In File: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.txt | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-07 18:47:17,462 : Parser_C : Writing to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-07 18:47:17,653 : Flow Logger HTML_K : Completed HTML Conversion To Json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-07 18:47:17,655 : Flow Logger HTML_K : Starting Json Split | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-07 18:47:17,661 : Style Dictionary_Q : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | Json | Adakveo-4874 EN PI.json
2021-05-07 18:47:17,717 : Style Dictionary_Q : Qrd Section Keys Retrieved For Style Dictionary: ANNEX I, ANNEX II, ANNEX III, B. PACKAGE LEAFLET | H | CAP |  en | Json | Adak

Starting Heading Extraction For File :- Adakveo-4874 EN PI_SmPC.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\en\Adakveo-4874 EN PI_SmPC.json
--------------------------------------------
SmPC


2021-05-07 18:47:18,906 : Heading Extraction Adakveo-4874 EN PI_SmPC.json : Started Extracting Heading | H | CAP |  en | 0 | Adakveo-4874 EN PI_SmPC.json
2021-05-07 18:47:19,355 : Heading Extraction Adakveo-4874 EN PI_SmPC.json : Match Passed | H | CAP |  en | 0 | Adakveo-4874 EN PI_SmPC.json | Doc txt :- 'SUMMARY OF PRODUCT CHARACTERISTICS' | Qrd txt :- 'SUMMARY OF PRODUCT CHARACTERISTICS' | Matched :- 'True'
2021-05-07 18:47:19,370 : Heading Extraction Adakveo-4874 EN PI_SmPC.json : Validation Passed As This The First Heading | H | CAP |  en | 0 | Adakveo-4874 EN PI_SmPC.json | currHeadId :- '20001' | currParentHeadId :- 'nan' | prevParentHeadId :- ''


...

 are for single use only.' | Qrd txt :- 'Pregnancy <and> <,> breast-feeding <and fertility>' | Matched :- 'False' en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Adakveo vials
 are for single use only.' | Qrd txt :- 'This leaflet was last revised in <{MM/YYYY}><{month YYYY}>.' | Matched :- 'False' Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Adakveo vials
2021-05-07 18:49:28,008 : Heading Extraction Adakveo-4874 EN PI_ PACKAGE LEAFLET.json : Match Failed | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Preparing the infusion' | Qrd txt :- 'Pregnancy <and> <,> breast-feeding <and fertility>' | Matched :- 'False'
2021-05-07 18:49:28,194 : Heading Extraction Adakveo-4874 EN PI_ PACKAGE LEAFLET.json : Match Failed | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Preparing the infusion' | Qrd txt :- 'This leaflet was last revised in <{MM/YYYY}><{month YYYY}>.' | Matched :- 'False'
 aseptic techniques.' | Qrd t

2021-05-07 18:49:39,453 : Heading Extraction Adakveo-4874 EN PI_ PACKAGE LEAFLET.json : Match Failed | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Storage of the diluted solution' | Qrd txt :- 'Pregnancy <and> <,> breast-feeding <and fertility>' | Matched :- 'False'
2021-05-07 18:49:39,570 : Heading Extraction Adakveo-4874 EN PI_ PACKAGE LEAFLET.json : Match Failed | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Storage of the diluted solution' | Qrd txt :- 'This leaflet was last revised in <{MM/YYYY}><{month YYYY}>.' | Matched :- 'False'
 overall.' | Qrd txt :- 'Pregnancy <and> <,> breast-feeding <and fertility>' | Matched :- 'False'led | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Chemical and
 overall.' | Qrd txt :- 'This leaflet was last revised in <{MM/YYYY}><{month YYYY}>.' | Matched :- 'False' CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Chemical and
2021-05-07 18



Heading Not Found 
 ['q This medicine is subject to additional monitoring. This will allow quick identification of new safety information. You can help by reporting any side effects you may get. See the end of section 4 for how to report side effects.', 'What X is and what it is used for', 'Do not <take> <use> X', 'Other medicines and X', 'X with <food> <and> <,> <drink> <and> <alcohol>', 'X contains {name the excipient(s)}', 'How to <take> <use> X ', 'Use in children <and adolescents>', 'If you <take> <use> more X than you should', 'If you forget to <take> <use> X>', 'If you stop <taking> <using> X>', 'Additional side effects in children <and adolescents>', 'How to store X', 'What X contains ', 'What X looks like and contents of the pack', 'Marketing Authorisation Holder and Manufacturer']


dict_keys(['q This medicine is subject to additional monitoring. This will allow quick identification of new safety information. You can help by reporting any side effects you may get. See the e

2021-05-07 18:49:42,789 : XmlGeneration_3 : Writing to File:Adakveo-4874 EN PI_ PACKAGE LEAFLET.xml | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json
2021-05-07 18:49:45,404 : XML Submission Logger_3 : Initiating Submission To FHIR Server | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json
2021-05-07 18:49:45,408 : XML Submission Logger_3 : POST sucessful: XML added with id: 47b4a54a-2137-4206-8200-6ca045b75671 | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json
2021-05-07 18:49:45,412 : Flow Logger HTML_K : Completed Processing Partitioned Jsons | H | CAP |  en | HTML | Adakveo-4874 EN PI.html


POST sucessful: XML added with id 47b4a54a-2137-4206-8200-6ca045b75671
Created XML File For :- Adakveo-4874 EN PI_ PACKAGE LEAFLET.json
